<a href="https://colab.research.google.com/github/ryan840919/STAS-Detection/blob/main/Prediction_and_Ground_true.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
def IOU(lis1,lis2):
  xmin_it = max(lis1[0],lis2[0])
  ymin_it = max(lis1[1],lis2[1])
  xmax_it = min(lis1[2],lis2[2])
  ymax_it = min(lis1[3],lis2[3])

  area_it = max(0,xmax_it-xmin_it+1)*max(0,ymax_it-ymin_it+1)

  area_1 = (lis1[2]-lis1[0]+1)*(lis1[3]-lis1[1]+1)
  area_2 = (lis2[2]-lis2[0]+1)*(lis2[3]-lis2[1]+1)
  
  IOU = area_it/(area_1+area_2-area_it)
  return IOU

## Generate the images labeled by prediction result and ground true.

In [ ]:
def P_and_G(test_name):
  # make a file of the recent test_name to prepare to store the labeled images about to generate.
  os.makedirs(f'/content/gdrive/MyDrive/AI競賽_運算/data/val_labeled/{test_name}/P_and_G')

  filename = os.listdir(f'/content/gdrive/MyDrive/AI競賽_運算/yolov5/yolov5/runs/val/{test_name}/labels')
  numlist = []
  for fil in filename:
    name = fil.split('.')
    numlist.append(int(name[0]))

  for num in numlist:
    # open and read the labels.txt files generated by the recent test.
    with open(f'/content/gdrive/MyDrive/AI競賽_運算/yolov5/yolov5/runs/val/{test_name}/labels/{num:08}.txt','r') as R:
      corlis = []
      for line in R.readlines():
        word = line.split(' ')
        xmin = round(float(word[1])*1716-float(word[3])*1716/2)
        ymin = round(float(word[2])*942-float(word[4])*942/2)
        xmax = round(float(word[1])*1716+float(word[3])*1716/2)
        ymax = round(float(word[2])*942+float(word[4])*942/2)
        score = float(word[5])
        cor = [xmin,ymax,xmax,ymin,score]
        corlis.append(cor)

    # open and read the labels.txt files that are initially given by the val set (ground true).
    with open(f'/content/gdrive/MyDrive/AI競賽_運算/data/STAS_YOLOv5_PS/labels/val/{num:08}.txt','r') as RL:
      corlisL = []
      for line in RL.readlines():
        wordL = line.split(' ')
        xminL = round(float(wordL[1])*1716-float(wordL[3])*1716/2)
        yminL = round(float(wordL[2])*942-float(wordL[4])*942/2)
        xmaxL = round(float(wordL[1])*1716+float(wordL[3])*1716/2)
        ymaxL = round(float(wordL[2])*942+float(wordL[4])*942/2)
        corL = [xminL,ymaxL,xmaxL,yminL]
        corlisL.append(corL)

    # read the val image files that are initially splited
    img = cv2.imread(f'/content/gdrive/MyDrive/AI競賽_運算/data/STAS_YOLOv5_PS/images/val/{num:08}.jpg')
    for cor in corlis:
      cv2.rectangle(img, (cor[0],cor[1]), (cor[2],cor[3]),(0,0,255),2)
      cv2.putText(img, f'{cor[4]}',(cor[0],cor[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    for corL in corlisL:
      cv2.rectangle(img, (corL[0],corL[1]), (corL[2],corL[3]),(0,255,0),2)
    
    # write the new labeled images that are generated by the recent test to the file.
    cv2.imwrite(f'/content/gdrive/MyDrive/AI競賽_運算/data/val_labeled/{test_name}/P_and_G/{num:08}.jpg',img)
    # cv2_imshow(img)

In [ ]:
P_and_G('test_for_label')

In [ ]:
P_and_G('test_with_background')

In [ ]:
P_and_G('test_with_background_lr0')

## Generate the images that are false-positive.

In [ ]:
def False_Positive(test_name):
  os.makedirs(f'/content/gdrive/MyDrive/AI競賽_運算/data/val_labeled/{test_name}/False_Positive')
  filename = os.listdir(f'/content/gdrive/MyDrive/AI競賽_運算/yolov5/yolov5/runs/val/{test_name}/labels')
  numlist = [int(fil.split('.')[0]) for fil in filename]
  numlist.sort()
  for num in numlist:
    im = cv2.imread(f'/content/gdrive/MyDrive/AI競賽_運算/data/STAS_YOLOv5_PS/images/val/{num:08}.jpg')
    with open(f'/content/gdrive/MyDrive/AI競賽_運算/yolov5/yolov5/runs/val/{test_name}/labels/{num:08}.txt','r') as R:
      corlis = []
      for line in R.readlines():
        word = line.split(' ')
        xmin = round(float(word[1])*1716-float(word[3])*1716/2)
        ymin = round(float(word[2])*942-float(word[4])*942/2)
        xmax = round(float(word[1])*1716+float(word[3])*1716/2)
        ymax = round(float(word[2])*942+float(word[4])*942/2)
        score = float(word[5])
        cor = [xmin,ymin,xmax,ymax,score]
        corlis.append(cor)

    with open(f'/content/gdrive/MyDrive/AI競賽_運算/data/STAS_YOLOv5_PS/labels/val/{num:08}.txt','r') as RL:
      corlisL = []
      for line in RL.readlines():
        wordL = line.split(' ')
        xminL = round(float(wordL[1])*1716-float(wordL[3])*1716/2)
        yminL = round(float(wordL[2])*942-float(wordL[4])*942/2)
        xmaxL = round(float(wordL[1])*1716+float(wordL[3])*1716/2)
        ymaxL = round(float(wordL[2])*942+float(wordL[4])*942/2)
        corL = [xminL,yminL,xmaxL,ymaxL]
        corlisL.append(corL)
    count = 0
    for cor in corlis:
      if not any(IOU(cor,corL)>0.5 for corL in corlisL):
        count += 1
        pim = im[cor[1]:cor[3],cor[0]:cor[2]]
        cv2.imwrite(f'/content/gdrive/MyDrive/AI競賽_運算/data/val_labeled/{test_name}/False_Positive/{num:08}_{count}.jpg',pim)
        with open(f'/content/gdrive/MyDrive/AI競賽_運算/data/val_labeled/{test_name}/False_Positive/{num:08}_{count}.txt','w') as S:
          S.write(f'{cor[4]}')

In [ ]:
False_Positive('test_for_label')

In [ ]:
False_Positive('test_with_background')

In [ ]:
False_Positive('test_with_background_lr0')

## Generate the images that are flase_negative.

In [ ]:
def False_Negative(test_name):
  os.makedirs(f'/content/gdrive/MyDrive/AI競賽_運算/data/val_labeled/{test_name}/False_Negative')
  filename = os.listdir(f'/content/gdrive/MyDrive/AI競賽_運算/yolov5/yolov5/runs/val/{test_name}/labels')
  numlist = [int(fil.split('.')[0]) for fil in filename]
  numlist.sort()
  for num in numlist:
    im = cv2.imread(f'/content/gdrive/MyDrive/AI競賽_運算/data/STAS_YOLOv5_PS/images/val/{num:08}.jpg')
    with open(f'/content/gdrive/MyDrive/AI競賽_運算/yolov5/yolov5/runs/val/{test_name}/labels/{num:08}.txt','r') as R:
      corlis = []
      for line in R.readlines():
        word = line.split(' ')
        xmin = round(float(word[1])*1716-float(word[3])*1716/2)
        ymin = round(float(word[2])*942-float(word[4])*942/2)
        xmax = round(float(word[1])*1716+float(word[3])*1716/2)
        ymax = round(float(word[2])*942+float(word[4])*942/2)
        score = float(word[5])
        cor = [xmin,ymin,xmax,ymax,score]
        corlis.append(cor)

    with open(f'/content/gdrive/MyDrive/AI競賽_運算/data/STAS_YOLOv5_PS/labels/val/{num:08}.txt','r') as RL:
      corlisL = []
      for line in RL.readlines():
        wordL = line.split(' ')
        xminL = round(float(wordL[1])*1716-float(wordL[3])*1716/2)
        yminL = round(float(wordL[2])*942-float(wordL[4])*942/2)
        xmaxL = round(float(wordL[1])*1716+float(wordL[3])*1716/2)
        ymaxL = round(float(wordL[2])*942+float(wordL[4])*942/2)
        corL = [xminL,yminL,xmaxL,ymaxL]
        corlisL.append(corL)
    count = 0
    for corL in corlisL:
      if not any(IOU(corL,cor)>0.5 for cor in corlis):
        count += 1
        pim = im[corL[1]:corL[3],corL[0]:corL[2]]
        cv2.imwrite(f'/content/gdrive/MyDrive/AI競賽_運算/data/val_labeled/{test_name}/False_Negative/{num:08}_{count}.jpg',pim)

In [ ]:
False_Negative('test_for_label')

In [ ]:
False_Negative('test_with_background')

In [ ]:
False_Negative('test_with_background_lr0')